In [1]:
import numpy as np
import pandas as pd
from tableone import TableOne

import sys
sys.path.append('src/')
import tir as tir

## Load dataSample

An input dataset for *tir* is a dataframe containing time to each CGM readings along with time-dependent or time-independent covariates of interest. The input dataset should take long format, where
each row contains the information in one time interval in which a CGM is observed. As such, multiple rows in the dataset may correspond to one subject. 

*dataSample* is a simulated dataset perturbated from a real dataset. *dataSample* contains 50 subjects. For each subject, *dataSample* contains values for the following eight variables: *patient_id*, which is the unique identifier for each subject; *glucose*, which is the CGM reading; *time*, which records the time in minutes reading CGM; x1, which is a binary covariate; x2, which is a continuous covariate.


In [2]:
data = pd.read_csv('data/dataSample.csv')
data.head()

,patient_id,glucose,time,x1,x2
0,EM016,87,0,1,0.606429
1,EM016,90,5,1,0.606429
2,EM016,93,10,1,0.606429
3,EM016,97,15,1,0.606429
4,EM016,103,20,1,0.606429


## Baseline characteristics

We can check the baseline characteristics of *dataSample* using *tableone()* function from python package *TableOne*

In [3]:
data_baseline = data.groupby('patient_id').first().reset_index()
columns = ['x1', 'x2']
categorical = ['x1']
table = TableOne(data_baseline, columns=columns, categorical=categorical)
table

Missing     Overall
n                                41
x1, n (%)     0       0   19 (46.3)
              1           22 (53.7)
x2, mean (SD)         0  -0.0 (1.0)

## Main function for calculating mean TIR
Function *estTIR()* in our package is the main function to calculate mean TIR. It has following arguments:

- data: A long format data frame. An example dataframe is 'dataSample'.
- time: A range of time for calculating TIR. '[a,b]' represents calculating TIR between time range from a to b.
- range: A range of target glucose values. '[70,180]' represents calculating TIR 70-180 mg/dL.
- method: Indicating the method used for calculating TIR. Values can be 'naive' and 'proposed' (default).
- model: Used only when method = 'proposed'. Indicating the model for follow-up duration. Values can be 'NULL' (non-informative, default) and 'cox' (informative; Cox-modeled follow-up duration.) 
- formula: Indicating the covariates for the Cox's model. The formula has the format like "x1+x2".
- boot: Number of bootstrap replicates used to obtain the standard error estimation. The default is 'NULL' which indicates bootstrap is not conducted.
- id: Indicating the column name of the subject identifier in data. The default is 'patient_id'.
- time_col: Indicating the column name of the time for reading CGM. The default is 'time'.
- glucose: Indicating the column name of CGM readings. The default is 'glucose'.
- period: Indicating the period of reading CGM. The default is 5 (minutes).

Function *printTIR()* summary the results returned by *estTIR()*. If 'boot' is specified in *estTIR()*, *printTIR()* shows:

- est: mean TIR estimation from the given dataset.
- std err: standard error for mean TIR.
- [0.025 0.975]: $95\%$ confidence interval of mean TIR estimation.

If 'boot' is not specified, *printTIR()* will only print mean TIR estimation.

### Calculating mean TIR with naive method. 
The following code is used to calculate mean TIR by the naive method with target glucose range 70-180mg/dL, time window 0-3 days (0-4200 minutes)，and 20 bootstrap replicates.

In [6]:
est_naive = tir.estTIR(data, method = 'naive', time = [0, 4200], range = [70,180], boot = 20)

In [7]:
tir.printTIR(est_naive)

      est  std err  [0.025  0.975]
TIR  0.57    0.049   0.494   0.646


### Calculate mean TIR with the proposed method under non-informative follow-up duration
The following code is used to calculate mean TIR by the proposed method under non-informative patient's follow-up duration assumption with glucose range 70-180mg/dL, time window 0-3 days (0-4200 minutes)，and 20 bootstrap replicates.

In [8]:
est_proposed_noninformative = tir.estTIR(data, method = 'proposed',time = [0, 4200], range = [70,180], boot = 20)

In [9]:
tir.printTIR(est_proposed_noninformative)

      est  std err  [0.025  0.975]
TIR  0.58    0.044   0.518   0.661


### Calculate mean TIR with the proposed method under Cox-model-based follow-up duration
The following code is used to calculate mean TIR by the proposed method under Cox-model-based follow-up duration with glucose range 70-180mg/dL, time window 0-3 days (0-4200 minutes)，20 bootstrap replicates, and covariates "x1" and "x2" with formula "x1+x2".

In [10]:
# A warning might be raised because of some coding style in lifelines to perform Cox regression.
# It will NOT affect the results.
# Here we ignore the warnings.
# Comment the following two lines if you want to see the warnings.
import warnings
warnings.filterwarnings('ignore')

est_proposed_cox = tir.estTIR(data, method = 'proposed', model = 'cox', formula = 'x1+x2', time = [0, 4200], range = [70,180], boot = 20)

In [11]:
tir.printTIR(est_proposed_cox)

       est  std err  [0.025  0.975]
TIR  0.581    0.032   0.523   0.634
